In [2]:
import numpy as np 
import pandas as pd 

In [14]:
real_data = pd.read_csv('True.csv')
fake_data = pd.read_csv('Fake.csv')
snopes = pd.read_csv('snopes_phase2_clean_2018_7_3.csv')
articles= pd.read_csv('news_articles.csv')

In [4]:
real_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [12]:
snopes.head()

,fact_rating_phase1,snopes_url_phase1,article_title_phase1,article_category_phase1,article_date_phase1,article_claim_phase1,article_origin_url_phase1,index_paragraph_phase1,page_is_first_citation_phase1,error_phase2,original_article_text_phase2,article_title_phase2,publish_date_phase2,author_phase2
0,true,https://www.snopes.com/fact-check/is-trump-adm...,Is the Trump Administration Paying Southwest K...,Politics Politics,22 June 2018,The Trump administration is paying Southwest K...,https://www.dallasnews.com/news/immigration/20...,1,True,No Error,"A Southwest Key spokeswoman, Cindy Casares, ha...",Texas nonprofit has received $1.5 billion in f...,2018-06-19,Jennifer Emily
1,true,https://www.snopes.com/fact-check/is-trump-adm...,Is the Trump Administration Paying Southwest K...,Politics Politics,22 June 2018,The Trump administration is paying Southwest K...,https://www.bloomberg.com/news/articles/2018-0...,1,False,No Error,The Trump administration plans to pay a Texas ...,Trumps Immigrant Child Detentions Mean $458 Mi...,2018-06-19,Alex Wayne
2,true,https://www.snopes.com/fact-check/is-trump-adm...,Is the Trump Administration Paying Southwest K...,Politics Politics,22 June 2018,The Trump administration is paying Southwest K...,https://web.archive.org/web/20180604042055/htt...,10,False,No Error,Who We Are Southwest Key Programs is a private...,Who We Are: Southwest Key Programs,NaN,NaN
3,true,https://www.snopes.com/fact-check/is-trump-adm...,Is the Trump Administration Paying Southwest K...,Politics Politics,22 June 2018,The Trump administration is paying Southwest K...,https://www.hhs.gov/about/budget/fy2018/budget...,10,False,No Error,Administration for Children and Families : Dis...,FY 2018 Budget in Brief - ACF - Discretionary,2017-05-19,NaN
4,true,https://www.snopes.com/fact-check/is-trump-adm...,Is the Trump Administration Paying Southwest K...,Politics Politics,22 June 2018,The Trump administration is paying Southwest K...,https://www.justice.gov/opa/pr/attorney-genera...,11,False,No Error,Attorney General Jeff Sessions today notified ...,Attorney General Announces Zero-Tolerance Poli...,NaN,NaN


In [15]:
articles.head()

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0
2,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1.0
3,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1.0
4,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...,1.0
